In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import findspark
findspark.init()

from pathlib import Path
import time

import h5py
import numpy as np
import pyqtgraph as pg
import pyspark

from janelia_core.fileio.exp_reader import find_images
from janelia_core.fileio.exp_reader import read_img_file

In [4]:
# Location of the images
image_folder = Path(r'E:\bishop_work\ahrens_two_color\20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950\processed\reg_images\neurons')

In [5]:
## Find our images
images = find_images(image_folder, '.h5')

Searching for image files...
Found 12500 images.


In [8]:
## Start a Spark Context
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '5g'), ('spark.executor.cores', '60'),
                                   ('spark.cores.max', '60'), ('spark.driver.memory','200g'), 
                                  ('spark.driver.maxResultSize', '200g')])
sc = pyspark.SparkContext(conf=conf)

In [16]:
def read_region(filepath):
    with h5py.File(filepath) as f:
        return f['data'][:, 0:200, 0:200]

In [17]:
t0 = time.time()
sc.parallelize(images).map(read_region).collect()
t1 = time.time()
print('Elapsed time: ' + str(t1-t0))

Elapsed time: 424.77157521247864
